In [ ]:
import json
import math
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tensorflow as tf
import gc
import itertools

import utils

from PIL import Image
from functools import partial
from sklearn import metrics
from collections import Counter
from keras import backend as K
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score

from keras import layers
from keras.applications import ResNet50, MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
import numpy as np
import json
import utils
import gc

from keras import backend as K
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

In [ ]:
benign_path = 'D:/Medical_Imaging_Projekt/Dataset/benign/'
malignant_path = 'D:/Medical_Imaging_Projekt/Dataset/malignant/'

In [ ]:
benign_images = np.array(utils.get_all_images(benign_path, 256))
malignant_images = np.array(utils.get_all_images(malignant_path, 256))

print(len(benign_images))
print(len(malignant_images))

In [ ]:
x_train, x_val, y_train, y_val = utils.create_split(benign_images, malignant_images)

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = x_train[0]
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(img.shape)

rgb_batch = np.repeat(img[..., np.newaxis], 3, -1)

print(rgb_batch.shape)

plt.imshow(rgb_batch)

## Display some Images

In [ ]:
utils.display_images(x_train, y_train)

## Data Generator

In [ ]:
train_generator = utils.create_train_generator()

## Model (DenseNet201)

In [ ]:
K.clear_session()
gc.collect()

model_densenet = utils.get_model("DenseNet", 256, 3)
model_resnet = utils.get_model("ResNet", 256, 3)
    
model = utils.build_model(model_densenet , lr = 1e-4)
model.summary()

In [ ]:
# Learning Rate Reducer
learn_control = utils.learning_rate_reducer()

filepath = "C:/Users/elias/OneDrive - FH Technikum Wien/Master_Technikum_Wien/3_Semester/Spezialisierung_Medical_Imaging/Projekt/Medical_Imaging_Projekt_2/weights.best.hdf5"
checkpoint = utils.model_checkpoint(filepath)


## Training and Evaluation

In [ ]:
history = utils.train_model(model, train_generator, x_train, y_train, x_val, y_val, learn_control, checkpoint)

In [ ]:
with open('history.json', 'w') as f:
    
    json.dump(str(history.history), f)

In [ ]:
utils.plot_metrics(history, 'accuracy', 'val_accuracy')

In [ ]:
utils.plot_metrics(history, 'loss', 'val_loss')

## Prediction

In [ ]:
model.load_weights(filepath)

# model = model.load_weights("weights.best.hdf5")

In [ ]:
Y_val_pred = model.predict(x_val)

In [ ]:
accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1))

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
tta_steps = 10
predictions = []

for i in tqdm(range(tta_steps)):

    preds = model.predict(train_generator.flow(X_test, batch_size=BATCH_SIZE, shuffle=False),
                          steps = len(X_test)/BATCH_SIZE)
    
    predictions.append(preds)
    gc.collect()
    
Y_pred_tta = np.mean(predictions, axis=0)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=55)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cm = confusion_matrix(np.argmax(Y_test, axis = 1), np.argmax(Y_pred, axis = 1))

cm_plot_label = ['benign', 'malignant']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for Skin Cancer')

In [ ]:
cm = confusion_matrix(np.argmax(Y_test, axis = 1), np.argmax(Y_pred_tta, axis=1))

cm_plot_label = ['benign', 'malignant']
plot_confusion_matrix(cm, cm_plot_label, title ='Confusion Metrix for Skin Cancer')

## Classification Report

In [ ]:
from sklearn.metrics import classification_report
classification_report(np.argmax(Y_test, axis=1), np.argmax(Y_pred_tta, axis=1))

## ROC and AUC

In [ ]:
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve
roc_log = roc_auc_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred_tta, axis=1))
false_positive_rate, true_positive_rate, threshold = roc_curve(np.argmax(Y_test, axis=1), np.argmax(Y_pred_tta, axis=1))
area_under_curve = auc(false_positive_rate, true_positive_rate)

plt.plot([0, 1], [0, 1], 'r--')
plt.plot(false_positive_rate, true_positive_rate, label='AUC = {:.3f}'.format(area_under_curve))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
#plt.savefig(ROC_PLOT_FILE, bbox_inches='tight')
plt.close()

In [ ]:
i = 0
prop_class=[]
mis_class=[]

for i in range(len(Y_test)):

    if(np.argmax(Y_test[i])==np.argmax(Y_pred_tta[i])):

        prop_class.append(i)

    if(len(prop_class)==8):

        break

i = 0

for i in range(len(Y_test)):

    if(not np.argmax(Y_test[i])==np.argmax(Y_pred_tta[i])):

        mis_class.append(i)

    if(len(mis_class)==8):

        break

# # Display first 8 images of benign
w = 60
h = 40
fig = plt.figure(figsize = (18, 10))
columns = 4
rows = 2

def Transfername(namecode):
    
    if namecode==0:
    
        return "Benign"
    
    else:
    
        return "Malignant"
    
for i in range(len(prop_class)):
    
    ax = fig.add_subplot(rows, columns, i+1)
    ax.set_title("Predicted result:"+ Transfername(np.argmax(Y_pred_tta[prop_class[i]]))
                       +"\n"+"Actual result: "+ Transfername(np.argmax(Y_test[prop_class[i]])))
    plt.imshow(X_test[prop_class[i]], interpolation='nearest')
    
plt.show()